In [4]:
from lightfm import LightFM
import configparser
import pandas as pd
from sklearn.model_selection import train_test_split
from lightfm.data import Dataset
from tqdm import tqdm
import numpy as np

## Data

In [5]:
# config
config = configparser.ConfigParser()
config.read('C:/Users/USER/Documents/github_personal/study/07_AI/lightfm/core/config.ini')
config['path']['dir.model.trained']
train_dataset_dir = config['path']['dir.dataset.train']
base_path = 'C:/Users/USER/Documents/github_personal/study/07_AI/lightfm'

In [6]:
# data 2 : 선별 데이터
dataset_name = 'ml-1m-csv-custom'
user_df = pd.read_csv(base_path + '/' + train_dataset_dir + '/' + dataset_name + '/users.csv', index_col=0)
item_df = pd.read_csv(base_path + '/' + train_dataset_dir + '/' + dataset_name + '/movies.csv', index_col=0)
rating_df = pd.read_csv(base_path + '/' + train_dataset_dir + '/' + dataset_name + '/ratings.csv', index_col=0)

In [7]:
# data 1 : 전체 데이터
# dataset_name = 'ml-1m-csv'
# user_df = pd.read_csv(base_path + '/' + train_dataset_dir + '/' + dataset_name + '/users.csv', index_col=0)
# item_df = pd.read_csv(base_path + '/' + train_dataset_dir + '/' + dataset_name + '/movies.csv', index_col=0)
# rating_df = pd.read_csv(base_path + '/' + train_dataset_dir + '/' + dataset_name + '/ratings.csv', index_col=0)

In [8]:
# make_train_data
total_df = rating_df.merge(user_df, how='left', on='user_idx')
total_df = total_df.merge(item_df, how='left', on='movie_idx')

In [9]:
# train data 중 일부만 사용
train_df = total_df.sample(100000)

In [10]:
# train_test_split
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

## Train

In [11]:
# id columns
USER_ID = 'user_idx'
ITEM_ID = 'movie_idx'
RATING = 'rating'

# user feature cols
if dataset_name == 'ml-1m-csv':
    user_feature_cols = ['gender', 'age', 'occupation']
    item_feature_cols = ['genres', 'release_date', 'original_language', 'adult',
                        'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
                        'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
                        'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
elif dataset_name == 'ml-1m-csv-custom':
    user_feature_cols = ['gender', 'age', 'occupation']
    item_feature_cols = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
                        'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
                        'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# make dataset
dataset = Dataset(
    user_identity_features = user_feature_cols,
    item_identity_features = item_feature_cols
)

# 데이터 fit : 모든 유저, 아이템, 피처의 고유값을 매핑
dataset.fit(
    users = (total_df[USER_ID]).astype(str).unique(),
    items = (total_df[ITEM_ID].astype(str).unique()),
    user_features = total_df[user_feature_cols].apply(lambda x: x.astype(str)).values.flatten(),
    item_features = total_df[item_feature_cols].apply(lambda x: x.astype(str)).values.flatten(),
    # user_features = user_feature_cols,
    # item_features = item_feature_cols
)

# check
num_users, num_items = dataset.interactions_shape()
print(f"Number of users: {num_users}, Number of items: {num_items}")

Number of users: 6040, Number of items: 3666


In [12]:
# create matrix
# 상호작용 및 가중치 행렬 생성 (훈련 데이터 사용)
(interactions, weights) = dataset.build_interactions(
    (str(row[USER_ID]), str(row[ITEM_ID]), row[RATING])
    for index, row in train_df.iterrows()
)

# 테스트 데이터에 대해서도 상호작용 행렬만 생성
(test_interactions, _) = dataset.build_interactions(
    (str(row[USER_ID]), str(row[ITEM_ID]), row[RATING])
    for index, row in train_df.iterrows()
)

In [33]:
# 유저 피처 행렬 생성
# 각 유저 ID에 대해 해당 유저의 모든 피처 값(성별, 연령, 직업 등)을 리스트로 묶어 전달합니다.
user_features_data = []
# 유저 ID의 고유 목록을 가져옵니다.
unique_users = train_df[USER_ID].astype(str).unique()
for user_id in tqdm(unique_users):
    user_row = train_df[train_df[USER_ID].astype(str) == user_id].iloc[0]
    features = []
    for col in user_feature_cols:
        features.append(str(user_row[col])) # 모든 피처를 문자열로 변환

    user_features_data.append((user_id, features))

user_features = dataset.build_user_features(user_features_data)

# 아이템 피처 행렬 생성
# 각 아이템 ID에 대해 해당 아이템의 모든 피처 값(장르, 언어, 성인 여부 등)을 리스트로 묶어 전달합니다.
item_features_data = []
unique_items = train_df[ITEM_ID].astype(str).unique()
for item_id in tqdm(unique_items):
    item_row = train_df[train_df[ITEM_ID].astype(str) == item_id].iloc[0]
    features = []
    for col in item_feature_cols:
        # 원-핫 인코딩된 장르의 경우, 값이 1인 피처 이름만 추가 (LightFM은 피처 이름을 기반으로 매핑)
        # if col in ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']:
        # if col in item_feature_cols:
        #     if item_row[col] == 1:
        #         features.append(col)
        # else: # 기타 범주형 피처 (release_date, original_language, adult)는 값 자체를 추가
            # features.append(str(item_row[col]))
        features.append(str(item_row[col]))

    item_features_data.append((item_id, features))

item_features = dataset.build_item_features(item_features_data)

print(f"User features shape: {user_features.shape}")
print(f"Item features shape: {item_features.shape}")

  0%|          | 0/5880 [00:00<?, ?it/s]

100%|██████████| 3218/3218 [01:44<00:00, 30.85it/s]

User features shape: (6040, 6065)
Item features shape: (3666, 3667)


In [34]:
# -----------------------------------------------------------
# LightFM 모델 학습
# -----------------------------------------------------------

# LightFM 모델 초기화
# loss='warp' (Weighted Approximate-Rank Pairwise)는 Implicit Feedback에 적합
# loss='logistic' (Logistic)은 Explicit Feedback에 적합 (평점을 가중치로 사용할 때)
# no_components: 잠재 요인의 수 (차원)
model = LightFM(
    loss='logistic',  # Explicit Feedback (별점) 사용
    no_components=30, # 잠재 요인의 차원
    learning_rate=0.05,
    item_alpha=1e-6,  # 아이템 피처에 대한 정규화
    user_alpha=1e-6   # 유저 피처에 대한 정규화
)

# 모델 학습
# interactions: 훈련 상호작용 행렬
# sample_weight: rating (별점) 가중치 행렬
# user_features: 유저 메타데이터 행렬
# item_features: 아이템 메타데이터 행렬
model.fit(
    interactions=interactions,
    sample_weight=weights,  # 별점 가중치 사용
    user_features=user_features,
    item_features=item_features,
    epochs=20, # 학습 반복 횟수
    num_threads=2, # 병렬 처리 스레드 수
    verbose=True
)

print("LightFM 모델 학습 완료.")

Epoch: 100%|██████████| 20/20 [00:17<00:00,  1.13it/s]

LightFM 모델 학습 완료.


## Predict

In [59]:
# 예측을 원하는 유저 ID (예: 1번 유저)와 아이템 ID를 LightFM이 사용할 수 있도록 문자열로 변환합니다.
# 주의: 이 유저 ID는 'full_df'에 존재해야 합니다.
user_id_to_predict = '668'

In [60]:
# Dataset 객체에서 ID 매핑을 가져옵니다.
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

# 인덱스를 ID로 다시 변환하기 위한 역 매핑을 만듭니다.
item_inverse_map = {v: k for k, v in item_id_map.items()}

# 1. 모든 아이템의 내부 인덱스 (0부터 num_items-1까지)를 준비합니다.
n_items = interactions.shape[1]
all_item_indices = np.arange(n_items)

# 2. 특정 유저의 내부 인덱스를 가져옵니다.
# 유저 ID가 매핑에 없으면 KeyError가 발생할 수 있습니다.
try:
    user_internal_id = user_id_map[user_id_to_predict]
except KeyError:
    print(f"Error: User ID {user_id_to_predict} not found in mapping.")
    # 실제 사용 시, 여기서 예외 처리나 기본값 설정을 해야 합니다.
    user_internal_id = -1 # 임시 값 설정

if user_internal_id != -1:
    # 3. 유저가 이미 상호작용한 아이템 (학습 데이터 기준)을 확인하여 제외합니다.
    # interactions 행렬에서 해당 유저의 행을 가져와서 0보다 큰 값(상호작용)을 가진 열 인덱스를 찾습니다.
    known_positives = interactions.getrow(user_internal_id).indices
    
    # 추천 후보 목록: 전체 아이템 인덱스에서 이미 상호작용한 아이템을 제외합니다.
    recommend_item_indices = np.setdiff1d(all_item_indices, known_positives)
else:
    # 유저 ID가 없는 경우, 모든 아이템을 추천 후보로 설정하거나 종료합니다.
    recommend_item_indices = all_item_indices

In [61]:
# --- 모델 추론 ---
# model.predict(유저 내부 인덱스, 아이템 내부 인덱스 배열, 유저 피처, 아이템 피처)
scores = model.predict(
    user_internal_id, 
    recommend_item_indices, 
    user_features=user_features, 
    item_features=item_features
)

# --- 상위 N개 추천 ---
N_RECOMMENDATIONS = 10 

# 점수를 내림차순으로 정렬하여 상위 N개의 인덱스를 가져옵니다.
top_indices = np.argsort(-scores)[:N_RECOMMENDATIONS]

# 상위 N개의 아이템 인덱스와 예측 점수를 가져옵니다.
top_item_indices = recommend_item_indices[top_indices]
top_scores = scores[top_indices]

# LightFM 내부 인덱스를 원래의 movie_idx ID로 다시 변환합니다.
recommendations = [item_inverse_map[idx] for idx in top_item_indices]

print(f"\n--- User {user_id_to_predict} 평가(시청)한 영화 List ---")
# total_df[total_df['user_idx'] == int(user_id_to_predict)][['movie_idx', 'title', 'genres']]
for idx, row in total_df[total_df['user_idx'] == int(user_id_to_predict)].iterrows():
    history = f"Movie ID: {row['movie_idx']}, Title: {row['title']}, Genres: {row['genres']}"
    print(history)

print(f"\n--- User {user_id_to_predict} 을 위한 추천 Top {N_RECOMMENDATIONS} ---")
for movie_id, score in zip(recommendations, top_scores):
    # 실제 영화 제목(title)을 가져오려면 item_df를 사용해야 합니다.
    # 예시에서는 movie_idx만 출력합니다.
    movie_title = item_df.loc[item_df['movie_idx']==int(movie_id), 'title'].item()
    movie_genre = item_df.loc[item_df['movie_idx']==int(movie_id), 'genres'].item()
    result = f"Movie ID: {movie_id}, {movie_title}, {movie_genre} Predicted Score: {score:.4f}"
    print(result)


--- User 668 평가(시청)한 영화 List ---
Movie ID: 593, Title: Silence of the Lambs, The (1991), Genres: Drama|Thriller
Movie ID: 2073, Title: Fandango (1985), Genres: Comedy
Movie ID: 2094, Title: Rocketeer, The (1991), Genres: Action|Adventure|Sci-Fi
Movie ID: 3044, Title: Dead Again (1991), Genres: Mystery|Romance|Thriller
Movie ID: 780, Title: Independence Day (ID4) (1996), Genres: Action|Sci-Fi|War
Movie ID: 1610, Title: Hunt for Red October, The (1990), Genres: Action|Thriller
Movie ID: 2278, Title: Ronin (1998), Genres: Action|Crime|Thriller
Movie ID: 2424, Title: You've Got Mail (1998), Genres: Comedy|Romance
Movie ID: 1480, Title: Smilla's Sense of Snow (1997), Genres: Action|Drama|Thriller
Movie ID: 1653, Title: Gattaca (1997), Genres: Drama|Sci-Fi|Thriller
Movie ID: 1676, Title: Starship Troopers (1997), Genres: Action|Adventure|Sci-Fi|War
Movie ID: 3638, Title: Moonraker (1979), Genres: Action|Romance|Sci-Fi
Movie ID: 2699, Title: Arachnophobia (1990), Genres: Action|Comedy|Sci-Fi